### python .xml to .json

In [ ]:
!pip install xmltodict

In [ ]:
import json
import xmltodict

with open("/content/drive/MyDrive/LOTTE/LOTTE_sample/과자/10094_크라운)콘초66G/10094_0_m_10_meta.xml",'r') as f:
    xml_data = f.read()

# jsonString = json.dumps(xmltodict.parse(xml_data), indent=4)
json_data = xmltodict.parse(xml_data)

print(json_data['comp_cd']['div_cd']['item_cd'])

12920100001


### [Step0]: Lotte_sample Dataset Unzip

In [ ]:
%cd /content/drive/MyDrive/LOTTE_sample

/content/drive/MyDrive/LOTTE_sample


In [ ]:
!unzip -qq "/content/drive/MyDrive/LOTTE_sample/LOTTE_sample.zip"

### [Step1]: Lotte_sample -> original_data 

In [ ]:
!pip install xmltodict

In [ ]:
import shutil
import json
import xmltodict
import glob
import os

parent_path = '/content/drive/MyDrive/LOTTE/LOTTE_sample'
objects_path =  glob.glob(parent_path + '/*/*')

annotation_path = '/content/drive/MyDrive/LOTTE/original_data/Training/annotation'
images_path = '/content/drive/MyDrive/LOTTE/original_data/Training/images'

for path in objects_path: # 상품 한종류씩
    pathes = glob.glob(path + '/*')

    # 확장자 별 분류
    jpg_list = [file for file in pathes if file.endswith(".jpg")]
    xml_list = [file for file in pathes if file.endswith(".xml")]
    meta_xml_list = [file for file in pathes if file.endswith("_meta.xml")] # item_cd 추출 위함

    with open(meta_xml_list[0],'r') as f:
        xml_data = f.read()
    json_data = xmltodict.parse(xml_data)
    
    file_name = json_data['comp_cd']['div_cd']['item_cd']
    os.mkdir(annotation_path + '/' + file_name)  # 해당 상품 'item_cd'로 폴더 생성 
    os.mkdir(images_path + '/' + file_name)  # 해당 상품 'item_cd'로 폴더 생성 

    # .jpg 파일 옮기기 
    for path in jpg_list:
        shutil.move(path, images_path+'/'+file_name+'/'+path.split('/')[-1])
    # .xml, _meta.xml 파일 옮기기
    for path in xml_list:
        shutil.move(path, annotation_path+'/'+file_name+'/'+path.split('/')[-1])


### [Step2]: original_data -> augmented_data

In [ ]:
!python3 /content/drive/MyDrive/LOTTE/data_augmentation.py --root_path /content/drive/MyDrive/LOTTE/original_data/Training/ --save_path /content/drive/MyDrive/LOTTE/augmented_data2/Training/ --resize_ratio 0.4 --data_num 800 --num_process 4 --extract_edge 1

100% 200/200 [27:28<00:00,  8.24s/it]


### Draw GT Boxes from "LOTTE_annotation_No.json" file

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import json

def Draw_GT(original_image, path, GT_group):
    
    # Annotate -> object별 annotation
    annotated_image = original_image.convert('RGB')
    draw = ImageDraw.Draw(annotated_image)
    font = ImageFont.truetype("./calibril.ttf", 15)

    draw.text(xy=[10.0 ,10.0], text=str(path).upper(), fill='white',
                  font=font) ## 좌측 상단 해당 이미지 path
    
    ## Drawing GT Box !!
    # Suppress specific classes, if needed
    for i in range(len(GT_group)):

        # Boxes
        box_location = [GT_group[i][0], GT_group[i][1], GT_group[i][2], GT_group[i][3]]
        box_location = list(map(float, box_location))
        draw.rectangle(xy=box_location, outline='#176bfc')
        draw.rectangle(xy=[l + 1. for l in box_location], outline='#176bfc')  # a second rectangle at an offset of 1 pixel to increase line thickness
            
        # draw.rectangle(xy=[l + 2. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a third rectangle at an offset of 1 pixel to increase line thickness
        # draw.rectangle(xy=[l + 3. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a fourth rectangle at an offset of 1 pixel to increase line thickness

        # Text
        text_size = font.getsize('GT'.upper())
        text_location = [box_location[0] + 2., box_location[3]]
        textbox_location = [box_location[0], box_location[3] + text_size[1], box_location[0] + text_size[0] + 4.,
                            box_location[3]]
        draw.rectangle(xy=textbox_location, fill='#176bfc')
        draw.text(xy=text_location, text='GT'.upper(), fill='white',
                  font=font)
        
    del draw
  
    return annotated_image


if __name__ == '__main__':

    with open('/content/drive/MyDrive/LOTTE_annotation_169.json', 'r') as f:
        json_data = json.load(f)
    f.close()
    
    # f = open("/content/drive/MyDrive/RGBTDv2/test_split.txt", 'r')
    # lines = f.readlines()

    for idx, image_dict in enumerate(json_data):

        original_image = Image.open(image_dict['pathes'], mode='r')
        original_image = original_image.convert('RGB')
        Draw_GT(original_image, image_dict['pathes'], image_dict['boxes']).save('/content/drive/MyDrive/LOTTE_sample_GT/GT_image_' + str(idx+1) +'.jpg','JPEG')

        if idx % 20 == 0: # 얼마나 진행되었는지 확인용
          print(idx)
    print("Making GT boxes on Combination Images Complete!")


0
20
40
60
80
100
120
140
160
Making GT boxes on Combination Images Complete!


### Make Video from  GT annotated images

In [ ]:
import re
import cv2
import os
import numpy as np
import natsort ## 문자열로 된 숫자를 정렬하고 싶을 때!!

if __name__ == '__main__':
    path = '/content/drive/MyDrive/LOTTE_sample_GT' # "ims/2/a"
    paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".jpg$", i)]
    # paths = natsort.natsorted(paths)

    #pathIn= './ims/2/'
    pathOut = '/content/drive/MyDrive/LOTTE_samples_video.mp4'
    fps = 1

    frame_array = []
    for idx , path in enumerate(paths) :

        img = cv2.imread(path)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
